In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd /content/drive/My Drive/MachineLearningPractice/

/content/drive/My Drive/MachineLearningPractice


In [3]:
import torch.optim as optim
import torch.utils.data
import torchvision
import torch.nn as nn
import numpy as np
import argparse

In [4]:
!pip install import-ipynb
import import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=57fabf22229d4b2134cabf509d99c9c6fbcd260e42b95ec2a5b4b27dc0037f84
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [5]:
from models.alexnet import AlexNet
from tools.DataLoader import LoadData

importing Jupyter notebook from /content/drive/My Drive/MachineLearningPractice/models/alexnet.ipynb
importing Jupyter notebook from /content/drive/My Drive/MachineLearningPractice/tools/DataLoader.ipynb
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [6]:
class Solver(object):
  def __init__(self,config):
    self.ModelType = config.model
    self.model = None
    self.lr = config.lr
    self.epochs = config.epoch
    self.device = None
    self.DataType = "Cifar10"
    # self.train_batch_size = config.trainBatchSize
    # self.test_batch_size = config.testBatchSize
    self.criterion = None
    self.optimizer = None
    self.scheduler = None
  
  def DataLoading(self):
    dataset = LoadData(self.DataType)
    self.TrainSet = dataset['trainset']
    self.TestSet = dataset['testset']
    self.classes = dataset['classes']

  def ModelLoading(self):
    self.model = eval(self.ModelType)().to(self.device)
    self.optimizer = optim.SGD(self.model.parameters(),lr=self.lr,momentum=0.9,weight_decay=0.0006)
    # self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
    #scheduler
    self.criterion = nn.CrossEntropyLoss().to(self.device)
    
  def train(self,epoch):
    total_loss = 0
    total_correct = 0
    total_num = 0
    
    # kbar = pkbar.Kbar(target=len(self.TrainSet), epoch=epoch, num_epochs=self.epochs+1, width=10, always_stateful=False)
    for batch_num, (data, target) in enumerate(self.TrainSet):
      data, target = data.to(self.device), target.to(self.device)
      self.optimizer.zero_grad()
      output = self.model(data)
      loss = self.criterion(output, target)
      loss.backward()
      self.optimizer.step()
      total_loss += loss.item()
      prediction = torch.max(output, 1) #return a tuple
      total_num += target.size(0)
      # print("\n")
      # print(total_correct)
      # print("\n")
      # print((prediction[1] == target).int())
      total_correct += torch.sum((prediction[1] == target).int()).numpy()

      # kbar.update(batch_num,values=[("Loss",total_loss/(batch_num+1)),("Current Loss",loss.item()),("Acc",total_correct/total_num),("Correct",total_correct),("Sum",total_num)])
      if (batch_num%100==0):
          print("epoch:{:d} batch:{:d} loss:{:.4f} acc:{:.4f}({:d}/{:d})".format(epoch,batch_num,total_loss/(batch_num+1),total_correct/total_num,total_correct,total_num))
    return total_loss, total_correct/total_num

  def test(self,epoch):
    test_loss = 0
    test_correct = 0
    test_num = 0

    # kbar = pkbar.Kbar(target=len(self.TestSet), epoch=epoch, num_epochs=self.epochs+1, width=10, always_stateful=False)

    with torch.no_grad():
      for batch_num, (data, target) in enumerate(self.TestSet):
        data, target = data.to(self.device), target.to(self.device)
        output = self.model(data)
        loss = self.criterion(output, target)
        test_loss += loss.item()
        prediction = torch.max(output, 1) #return a tuple
        test_num += target.size(0)
        test_correct += torch.sum((prediction[1] == target).int()).numpy()

        # kbar.update(batch_num,values=[("Loss",test_loss/(batch_num+1)),("Acc",test_correct/test_num),("Correct",test_correct),("Sum",test_num)])
        if (batch_num%100==0):
          print("epoch:{:d} batch:{:d} loss:{:.4f} acc:{:.4f}({:d}/{:d})".format(epoch,batch_num,test_loss/(batch_num+1),test_correct/test_num,test_correct,test_num))
    return test_loss, test_correct/test_num

  def save(self):
    model_path = "model.pth"
    torch.save(self.model,model_path)
    print("Saved in {}".format(model_path))

  # load model data??

  def run(self):
    self.DataLoading()
    self.ModelLoading()
    accuracy = 0
    for epoch in range(1,self.epochs+1):
      # scheduler
      train_result = self.train(epoch)
      print(train_result)
      test_result = self.test(epoch)
      best_accuracy = max(accuracy,test_result[1])
      if epoch == self.epochs:
        print("\n==>Best Performance: %.3f%%"%(accuracy*100))
        self.save()

In [7]:
def main():
  parser = argparse.ArgumentParser(description="cifar-10 with PyTorch")
  parser.add_argument('--model',default='AlexNet',type=str,help='model type for training')
  parser.add_argument('--lr',default=0.005,type=float,help='learning rate')
  parser.add_argument('--epoch',default=3,type=int,help="epochs for training")
  parser.add_argument('--trainBatchSize',default=100,type=int,help="batch size for training")
  parser.add_argument('--testBatchSize',default=100,type=int,help="batche size for test")
  parser.add_argument('--cuda',default=torch.cuda.is_available(),type=bool,help='whether cuda is in use')
  parser.add_argument('-f')
  args = parser.parse_args()

  solver = Solver(args)
  solver.run()

In [8]:
main()

Files already downloaded and verified
Files already downloaded and verified
epoch:1 batch:0 loss:2.2980 acc:0.0000(0/4)
epoch:1 batch:100 loss:2.3038 acc:0.0965(39/404)
epoch:1 batch:200 loss:2.3037 acc:0.0908(73/804)
epoch:1 batch:300 loss:2.3063 acc:0.0814(98/1204)
epoch:1 batch:400 loss:2.3061 acc:0.0810(130/1604)
epoch:1 batch:500 loss:2.3054 acc:0.0893(179/2004)
epoch:1 batch:600 loss:2.3066 acc:0.0907(218/2404)
epoch:1 batch:700 loss:2.3059 acc:0.0924(259/2804)
epoch:1 batch:800 loss:2.3063 acc:0.0949(304/3204)
epoch:1 batch:900 loss:2.3062 acc:0.0938(338/3604)
epoch:1 batch:1000 loss:2.3056 acc:0.0949(380/4004)
epoch:1 batch:1100 loss:2.3053 acc:0.0972(428/4404)
epoch:1 batch:1200 loss:2.3054 acc:0.0985(473/4804)
epoch:1 batch:1300 loss:2.3047 acc:0.0999(520/5204)
epoch:1 batch:1400 loss:2.3023 acc:0.1030(577/5604)
epoch:1 batch:1500 loss:2.3004 acc:0.1066(640/6004)
epoch:1 batch:1600 loss:2.2986 acc:0.1067(683/6404)
epoch:1 batch:1700 loss:2.2987 acc:0.1066(725/6804)
epoch:1 ba

KeyboardInterrupt: ignored